In [6]:
import numpy as np
import matplotlib.pyplot as plt
import os
import subprocess

os.chdir("/Users/davidcoope/Desktop/BayesianOptimiser")
import BOPackage

In [7]:
def Ackley(X):
    n = X.shape[1]  # Number of dimensions (should be 20 for this problem)
    
    # Calculate the objective function value for each row in X
    term1 = -20 * np.exp(-0.2 * np.sqrt(np.sum(X**2, axis=1) / n))
    term2 = -np.exp(np.sum(np.cos(2 * np.pi * X), axis=1) / n)
    y = term1 + term2 + 20 + np.e

    # # Apply the constraints
    # constraint1 = np.sum(X, axis=1) > 0
    # constraint2 = np.linalg.norm(X, axis=1) - 5 > 0
    
    # # Apply penalties for constraint violations
    # y[constraint1 | constraint2] = np.inf
    
    # Make function negative to find the minimum
    y = -y.reshape(-1, 1)
    
    return y

bounds = bounds = np.array([[-5, 10]] * 20)
length_scale = 2 * np.pi
n_samples = 1000
ObjectiveFunction = Ackley

In [8]:
# X1 = np.linspace(bounds[0,0], bounds[0,1], 501, endpoint=True)
# X2 = np.linspace(bounds[1,0], bounds[1,1], 501, endpoint=True)

# # Create the meshgrid
# X1_grid, X2_grid = np.meshgrid(X1, X2)

# # Combine the grid into an n**2 x 2 matrix
# candidate_x = np.c_[X1_grid.ravel(), X2_grid.ravel()]

# result = ObjectiveFunction(candidate_x).reshape(X1_grid.shape)

# # Plot the contour
# contour = plt.contourf(X1, X2, result, levels=50)

# # Add a color bar
# plt.colorbar(contour)

# # Show the plot
# plt.show()

In [9]:
"""
Configure the optimiser.
"""
random_seed = 10

Kernel = BOPackage.RBF_Kernel

AcquisitionFunction = BOPackage.UpperConfidenceBound
max_kappa = 1
min_kappa = 0.05

reduce_bounds_factor = 0.5

log_path = '/Users/davidcoope/Desktop/BayesianOptimiser/Ackley/BOlog.log'

if os.path.exists(log_path):   
    subprocess.run(['rm', '-rf', log_path], check=True, text=True, capture_output=True)  

In [10]:
"""
Configure the optimiser for the standard configuration.
"""

dynamic_bounds = False

# Create the object
bo1 = BOPackage.BO(KernelFunction=Kernel, 
                  length_scale=length_scale, 
                  AcquisitionFunction=AcquisitionFunction, 
                  bounds=bounds, 
                  n_samples=n_samples,
                #   log_path=log_path,
                  dynamic_bounds=dynamic_bounds,
                  iterations_between_reducing_bounds=None,
                  first_reduce_bounds=None,
                  reduce_bounds_factor=reduce_bounds_factor,
                  random_seed=random_seed)

raw_X = bo1.GetRandomXBatch(1)
raw_y = ObjectiveFunction(raw_X)
bo1.UpdateData(raw_X, raw_y)

for i in range(299):
    raw_X = bo1.GetNextX(kappa=0.1)
    raw_y = ObjectiveFunction(np.array([raw_X]))
    bo1.UpdateData(raw_X, raw_y)

bo1.PrintCurrentStatus()
print(bo1.bounds)
# BOPackage.PlotData(bo1)

BO1 = bo1.BestData()[1][0]

Current best y value was [-9.16400027]; the corresponding X values were [ 0.95296975  1.4590439   0.84913755  5.43091299 -1.5715092   3.28813257
 -0.07201749 -4.08561293 -3.60211218 -0.75780125  1.60890745 -1.57593503
  0.19542201 -4.28497961  0.37402941 -0.84385613 -0.11677987  1.49398702
  1.18580759  0.79827554]
[[-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]]


In [11]:
"""
Configure the optimiser with bounds.
"""

dynamic_bounds = True
iterations_between_reducing_bounds = 30
first_reduce_bounds = 100  

# Create the object
bo2 = BOPackage.BO(KernelFunction=Kernel, 
                  length_scale=length_scale, 
                  AcquisitionFunction=AcquisitionFunction, 
                  bounds=bounds, 
                  n_samples=n_samples,
                #   log_path=log_path,
                  dynamic_bounds=dynamic_bounds,
                  iterations_between_reducing_bounds=iterations_between_reducing_bounds,
                  first_reduce_bounds=first_reduce_bounds,
                  reduce_bounds_factor=reduce_bounds_factor,
                  random_seed=random_seed)

raw_X = bo2.GetRandomXBatch(1)
raw_y = ObjectiveFunction(raw_X)
bo2.UpdateData(raw_X, raw_y)

for i in range(299):
    raw_X = bo2.GetNextX(kappa=0.1)
    raw_y = ObjectiveFunction(np.array([raw_X]))
    bo2.UpdateData(raw_X, raw_y)

bo2.PrintCurrentStatus()
print(bo2.bounds)
# BOPackage.PlotData(bo2)

BO2 = bo2.BestData()[1][0]

Current best y value was [-7.87812628]; the corresponding X values were [ 2.45836746  1.28163512 -2.25609371 -0.91355664 -1.24848859  2.0651416
  4.06932124  0.79084227  1.75838331  3.12422995  0.78182928 -0.05378453
 -0.47150383  0.7738714  -1.08371018  3.89776966  0.66308147 -1.57409408
  0.22806411  1.98389455]
The bounds have been reduced 3 times
[[ 1.64150037  3.51650037]
 [ 0.59656282  2.47156282]
 [-3.28057153 -1.40557153]
 [-2.41780924 -0.54280924]
 [-1.80983369  0.06516631]
 [ 1.12716431  3.00216431]
 [ 3.54515898  5.42015898]
 [ 0.17864181  2.05364181]
 [ 1.19393338  3.06893338]
 [ 2.11859232  3.99359232]
 [-0.19421211  1.68078789]
 [-0.94980404  0.92519596]
 [-1.100466    0.774534  ]
 [ 0.25479801  2.12979801]
 [-1.56199031  0.31300969]
 [ 2.30186625  4.17686625]
 [-0.76880918  1.10619082]
 [-2.23588148 -0.36088148]
 [-0.21929213  1.65570787]
 [ 1.02593828  2.90093828]]


In [12]:
"""
Configure the optimiser with batch sampling.
"""

dynamic_bounds = False

# Create the object
bo3 = BOPackage.BO(KernelFunction=Kernel, 
                  length_scale=length_scale, 
                  AcquisitionFunction=AcquisitionFunction, 
                  bounds=bounds, 
                  n_samples=n_samples,
                #   log_path=log_path,
                  dynamic_bounds=dynamic_bounds,
                  iterations_between_reducing_bounds=iterations_between_reducing_bounds,
                  first_reduce_bounds=first_reduce_bounds,
                  reduce_bounds_factor=reduce_bounds_factor,
                  random_seed=random_seed)

raw_X = bo3.GetRandomXBatch(10)
raw_y = ObjectiveFunction(raw_X)
bo3.UpdateData(raw_X, raw_y)

for i in range(29):
    raw_X = bo3.GetNextXBatch(10, sub_batch_size=10, max_kappa=max_kappa, min_kappa=min_kappa)
    raw_y = ObjectiveFunction(raw_X)
    bo3.UpdateData(raw_X, raw_y)

bo3.PrintCurrentStatus()
print(bo3.bounds)
# BOPackage.PlotData(bo3)

BO3 = bo3.BestData()[1][0]

Current best y value was [-9.16400027]; the corresponding X values were [ 0.95296975  1.4590439   0.84913755  5.43091299 -1.5715092   3.28813257
 -0.07201749 -4.08561293 -3.60211218 -0.75780125  1.60890745 -1.57593503
  0.19542201 -4.28497961  0.37402941 -0.84385613 -0.11677987  1.49398702
  1.18580759  0.79827554]
[[-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]]


In [13]:
"""
Configure the optimiser with sub-batch sampling and bounds reduction.
"""

dynamic_bounds = True
iterations_between_reducing_bounds = 3
first_reduce_bounds = 50 

# Create the object
bo6 = BOPackage.BO(KernelFunction=Kernel, 
                  length_scale=length_scale, 
                  AcquisitionFunction=AcquisitionFunction, 
                  bounds=bounds, 
                  n_samples=n_samples,
                #   log_path=log_path,
                  dynamic_bounds=dynamic_bounds,
                  iterations_between_reducing_bounds=iterations_between_reducing_bounds,
                  first_reduce_bounds=first_reduce_bounds,
                  reduce_bounds_factor=reduce_bounds_factor,
                  random_seed=random_seed)

raw_X = bo6.GetRandomXBatch(10)
raw_y = ObjectiveFunction(raw_X)
bo6.UpdateData(raw_X, raw_y)

for i in range(29):
    raw_X = bo6.GetNextXBatch(10, max_kappa=max_kappa, min_kappa=min_kappa)
    raw_y = ObjectiveFunction(raw_X)
    bo6.UpdateData(raw_X, raw_y)

bo6.PrintCurrentStatus()
print(bo6.bounds)
# BOPackage.PlotData(bo5)

BO6 = bo6.BestData()[1][0]

Current best y value was [-9.16400027]; the corresponding X values were [ 0.95296975  1.4590439   0.84913755  5.43091299 -1.5715092   3.28813257
 -0.07201749 -4.08561293 -3.60211218 -0.75780125  1.60890745 -1.57593503
  0.19542201 -4.28497961  0.37402941 -0.84385613 -0.11677987  1.49398702
  1.18580759  0.79827554]
The bounds have been reduced 0 times
[[-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]]


In [14]:
"""
Configure the optimiser with sub-batch sampling.
"""

dynamic_bounds = False

# Create the object
bo4 = BOPackage.BO(KernelFunction=Kernel, 
                  length_scale=length_scale, 
                  AcquisitionFunction=AcquisitionFunction, 
                  bounds=bounds, 
                  n_samples=n_samples,
                #   log_path=log_path,
                  dynamic_bounds=dynamic_bounds,
                  iterations_between_reducing_bounds=iterations_between_reducing_bounds,
                  first_reduce_bounds=first_reduce_bounds,
                  reduce_bounds_factor=reduce_bounds_factor,
                  random_seed=random_seed)

raw_X = bo4.GetRandomXBatch(10)
raw_y = ObjectiveFunction(raw_X)
bo4.UpdateData(raw_X, raw_y)

for i in range(29):
    raw_X = bo4.GetNextXBatch(10, sub_batch_size=2, max_kappa=max_kappa, min_kappa=min_kappa)
    raw_y = ObjectiveFunction(raw_X)
    bo4.UpdateData(raw_X, raw_y)

bo4.PrintCurrentStatus()
print(bo4.bounds)
# BOPackage.PlotData(bo4)

BO4 = bo4.BestData()[1][0]

Current best y value was [-9.22578324]; the corresponding X values were [ 3.86871141  1.41849967 -2.4937375   0.04800667 -3.93665924 -1.45817607
 -3.59630046  1.1069933   0.0381924   1.7746237  -0.04754627  0.80108958
 -2.03013026  5.42937771  2.35885507 -0.97284701  2.08031527  0.14275484
  2.42167942 -2.68051797]
[[-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]
 [-5 10]]


In [15]:
"""
Configure the optimiser with sub-batch sampling and bounds reduction.
"""

dynamic_bounds = True
iterations_between_reducing_bounds = 10
first_reduce_bounds = 30

# Create the object
bo5 = BOPackage.BO(KernelFunction=Kernel, 
                  length_scale=length_scale, 
                  AcquisitionFunction=AcquisitionFunction, 
                  bounds=bounds, 
                  n_samples=n_samples,
                  log_path=log_path,
                  dynamic_bounds=dynamic_bounds,
                  iterations_between_reducing_bounds=iterations_between_reducing_bounds,
                  first_reduce_bounds=first_reduce_bounds,
                  reduce_bounds_factor=reduce_bounds_factor,
                  random_seed=random_seed)

raw_X = bo5.GetRandomXBatch(100)
raw_y = ObjectiveFunction(raw_X)
bo5.UpdateData(raw_X, raw_y)

for i in range(99):
    raw_X = bo5.GetNextXBatch(100, sub_batch_size=20, max_kappa=max_kappa, min_kappa=min_kappa)
    raw_y = ObjectiveFunction(raw_X)
    bo5.UpdateData(raw_X, raw_y)
    

bo5.PrintCurrentStatus()
print(bo5.bounds)
# BOPackage.PlotData(bo5)

BO5 = bo5.BestData()[1][0]

In [ ]:
print(BO1, BO2, BO3, BO6, BO4, BO5)

[-inf] [-inf] [-inf] [-inf] [-inf] [-inf]


((0,), (array([-inf]),))
